In [176]:
import sys, os

import igl
import open3d as o3d
from meshplot import plot

import scipy as sp
import numpy as np
from sklearn.neighbors import KDTree

## Part 1

In [196]:
# utility function
def show_pointcloud(points):
    pts_o3d = o3d.geometry.PointCloud()
    pts_o3d.points = o3d.utility.Vector3dVector(points)
    o3d.visualization.draw_geometries([pts_o3d])

def evaluate_align(source_ply, target_ply, threshold=0.001):
    source_ptc = o3d.geometry.PointCloud()
    target_ptc = o3d.geometry.PointCloud()
    source_ptc.points = o3d.utility.Vector3dVector(np.asarray(source_ply.vertices))
    target_ptc.points = o3d.utility.Vector3dVector(np.asarray(target_ply.vertices))
    evaluation = o3d.pipelines.registration.evaluate_registration(source_ptc, target_ptc, threshold)
    return evaluation

### 1. Select a subset of points $p_i$

In [2]:
def uniform_sample(ply, n=1000):
    pcd = ply.sample_points_uniformly(number_of_points=n)
    p = pcd.points
    return np.asarray(p)

### 2. Match each $p_i$ to closest point $q_i$ on other scan

In [4]:
def match_points(sampled_pts, mesh_pts):
    n_pts, dim = sampled_pts.shape
    tree = KDTree(mesh_pts, leaf_size=2)
    dist, idx = tree.query(sampled_pts, k=1)
    return dist.reshape((n_pts,)), mesh_pts[idx].reshape((n_pts, dim))

### 3. Reject "bad" pairs $(p_i, q_i)$

In [5]:
def reject_pairs(p, q, dist):
    med = np.median(dist)
    idx = np.where(dist < 2. * med)
    return p[idx], q[idx]

### 4. Compute rotation $R$ and translation $t$

In [6]:
def solve_Rt(p, q):
    p_avg = np.mean(p, axis=0)
    q_avg = np.mean(q, axis=0)
    P = (p - p_avg).T
    Q = (q - q_avg).T
    u, s, vh = np.linalg.svd(Q @ P.T, full_matrices=True)
    R = vh.T @ np.array([
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., np.linalg.det(vh.T @ u.T)]
    ]) @ u.T
    t = p_avg - R @ q_avg
    return R, t

### 5. Iterate after scan alignment: $q_i \larr Rq_i + t$

In [168]:
def paint_overlap(ply1, ply2, radius=0.001):
    ply1_non_overlap_color = np.array([255, 127, 80]) / 255.
    overlap_color = np.array([138, 43, 226]) / 255.
    ply2_non_overlap_color = np.array([3, 168, 158]) / 255.
    
    points1 = np.asarray(ply1.vertices)
    points2 = np.asarray(ply2.vertices)
    colors1 = np.zeros((points1.shape[0], 3))
    colors2 = np.zeros((points2.shape[0], 3))

    tree1 = KDTree(points1, leaf_size=10) 
    tree2 = KDTree(points2, leaf_size=10) 

    has_neighbours1 = tree2.query_radius(points1, r=radius, count_only=True)
    colors1[has_neighbours1==0] = ply1_non_overlap_color
    colors1[has_neighbours1!=0] = overlap_color

    has_neighbours2 = tree1.query_radius(points2, r=radius, count_only=True)
    colors2[has_neighbours2==0] = ply2_non_overlap_color
    colors2[has_neighbours2!=0] = overlap_color

    ply1.vertex_colors=o3d.utility.Vector3dVector(colors1)
    ply2.vertex_colors=o3d.utility.Vector3dVector(colors2)

    return ply1, ply2

In [159]:
rng = np.random.RandomState(0)
X = rng.random_sample((10, 3))  # 10 points in 3 dimensions
Y = rng.random_sample((3, 3))
tree = KDTree(X, leaf_size=2)     
print(tree.query_radius(Y, r=0.3, count_only=True))
ind = tree.query_radius(Y, r=0.3)  
print(ind)  # indices of neighbors within distance 0.3

[1 0 1]
[array([3]) array([], dtype=int64) array([7])]


In [193]:
def icp(mesh1_fp, mesh2_fp, tl=1e-3, init_transform=None):
    # mesh2 to mesh1

    ply1 = o3d.io.read_triangle_mesh(mesh1_fp)
    ply2 = o3d.io.read_triangle_mesh(mesh2_fp)

    if init_transform is not None:
        ply2 = ply2.transform(init_transform)

    # ply1 = ply1.compute_vertex_normals()
    # ply2 = ply2.compute_vertex_normals()

    E = np.array([
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
    ])
    
    while True:
    
        points1 = np.asarray(ply1.vertices)
        sampled_points2 = uniform_sample(ply2)
        
        dist, corr_points1 = match_points(sampled_points2, points1)
        p, q = reject_pairs(corr_points1, sampled_points2, dist)
        R, t = solve_Rt(p, q)

        if np.linalg.norm(E - R) < tl:
            ply1, ply2 = paint_overlap(ply1, ply2)
            return ply1, ply2

        ply2 = ply2.rotate(R)
        ply2 = ply2.translate(t)


In [197]:
mesh1_fp = os.path.join('bunny_v2', 'bun000_v2.ply')
mesh2_fp = os.path.join('bunny_v2', 'bun045_v2.ply')
ply1, ply2 = icp(mesh1_fp, mesh2_fp)
o3d.io.write_triangle_mesh(os.path.join('bunny_v2', 'bun045_v2_reg_paint.ply'), ply2)
o3d.io.write_triangle_mesh(os.path.join('bunny_v2', 'bun000_v2_paint045.ply'), ply1)
evaluation = evaluate_align(ply2, ply1)
print(evaluation)

RegistrationResult with fitness=9.149812e-01, inlier_rmse=3.887063e-04, and correspondence_set size of 36688
Access transformation to get result.


In [198]:
mesh1_fp = os.path.join('bunny_v2', 'bun000_v2.ply')
mesh2_fp = os.path.join('bunny_v2', 'bun270_v2.ply')
ply1, ply2 = icp(mesh1_fp, mesh2_fp)
o3d.io.write_triangle_mesh(os.path.join('bunny_v2', 'bun000_v2_paint270.ply'), ply1)
o3d.io.write_triangle_mesh(os.path.join('bunny_v2', 'bun270_v2_reg_paint.ply'), ply2)
evaluation = evaluate_align(ply2, ply1)
print(evaluation)

RegistrationResult with fitness=9.570676e-02, inlier_rmse=6.400720e-04, and correspondence_set size of 3034
Access transformation to get result.


## Part 5

In [199]:
def convert_pts_to_ptc(pts):
    pts_o3d = o3d.geometry.PointCloud()
    pts_o3d.points = o3d.utility.Vector3dVector(pts)
    return pts_o3d

In [200]:
def preprocess_point_cloud(pcd_down, voxel_size):
    radius_normal = voxel_size * 2
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    radius_feature = voxel_size * 5
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd_down, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_fpfh

In [201]:
def prepare_dataset(voxel_size, mesh1_fp, mesh2_fp):
    ply1 = o3d.io.read_triangle_mesh(mesh1_fp)
    ply2 = o3d.io.read_triangle_mesh(mesh2_fp)
    source_down = convert_pts_to_ptc(uniform_sample(ply2, 10000))
    target_down = convert_pts_to_ptc(uniform_sample(ply1, 10000))

    source_fpfh = preprocess_point_cloud(source_down, voxel_size)
    target_fpfh = preprocess_point_cloud(target_down, voxel_size)
    return source_down, target_down, source_fpfh, target_fpfh

In [202]:
def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

In [203]:
mesh1_fp = os.path.join('bunny_v2', 'bun000_v2.ply')
mesh2_fp = os.path.join('bunny_v2', 'bun315_v2.ply')
voxel_size = 0.005
source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size, mesh1_fp, mesh2_fp)
result = execute_fast_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)
print(result.transformation)

[[ 7.15243179e-01 -1.58957941e-02 -6.98694868e-01 -7.26120028e-03]
 [ 2.20677025e-02  9.99756467e-01 -1.54787559e-04  2.57584837e-04]
 [ 6.98527173e-01 -1.53078798e-02  7.15419777e-01 -1.30532653e-02]
 [-0.00000000e+00  0.00000000e+00 -0.00000000e+00  1.00000000e+00]]


In [205]:
ply1, ply2 = icp(mesh1_fp, mesh2_fp, 5e-4, result.transformation)
o3d.io.write_triangle_mesh(os.path.join('bunny_v2', 'bun000_v2_paint315.ply'), ply1)
o3d.io.write_triangle_mesh(os.path.join('bunny_v2', 'bun315_v2_reg_paint.ply'), ply2)
evaluation = evaluate_align(ply2, ply1)
print(evaluation)

RegistrationResult with fitness=7.961286e-01, inlier_rmse=4.060414e-04, and correspondence_set size of 28132
Access transformation to get result.


## Part 6

In [14]:
def ptpl_match_points(sampled_pts, mesh_pts, n):
    n_pts, dim = sampled_pts.shape
    tree = KDTree(mesh_pts, leaf_size=2)
    dist, idx = tree.query(sampled_pts, k=1)
    return dist.reshape((n_pts,)), mesh_pts[idx].reshape((n_pts, dim)), n[idx].reshape((n_pts, dim))

In [15]:
def ptpl_reject_pairs(p, q, dist, n):
    med = np.median(dist)
    idx = np.where(dist < 2. * med)
    return p[idx], q[idx], n[idx]

In [208]:
def ptpl_solve_Rt(p, q, n_p):
    a = np.cross(q, n_p)
    A = np.hstack((a, n_p))
    b = np.array([np.sum(n_p * p - n_p * q, axis=1)]).T
    
    # u, s, vh = np.linalg.svd(A, full_matrices=False)
    # Aplus = vh.T @ np.linalg.pinv(np.diag(s)) @ u.T
    alpha, beta, gamma, tx, ty, tz = np.linalg.pinv(A.T @ A) @ A.T @ b

    R = np.array([
        [1., -gamma, beta],
        [gamma, 1., -alpha],
        [-beta, alpha, 1.]
    ])
    t = np.array([tx, ty, tz])
    return R, t

In [209]:
def ptpl_icp(mesh1_fp, mesh2_fp, tl=5e-4):
    # mesh2 to mesh1

    ply1 = o3d.io.read_triangle_mesh(mesh1_fp)
    ply2 = o3d.io.read_triangle_mesh(mesh2_fp)

    ply1 = ply1.compute_vertex_normals()
    n_p = np.asarray(ply1.vertex_normals)

    E = np.array([
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
    ])

    while True:
    
        points1 = np.asarray(ply1.vertices)
        sampled_points2 = uniform_sample(ply2)
        
        dist, corr_points1, n = ptpl_match_points(sampled_points2, points1, n_p)
        p, q, n = ptpl_reject_pairs(corr_points1, sampled_points2, dist, n_p)
        R, t = ptpl_solve_Rt(p, q, n)

        if np.linalg.norm(E - R) < tl:
            ply1, ply2 = paint_overlap(ply1, ply2)
            return ply1, ply2

        ply2 = ply2.rotate(R)
        ply2 = ply2.translate(t)


In [210]:
mesh1_fp = os.path.join('bunny_v2', 'bun000_v2.ply')
mesh2_fp = os.path.join('bunny_v2', 'bun045_v2.ply')
ply1, ply2 = ptpl_icp(mesh1_fp, mesh2_fp)
o3d.io.write_triangle_mesh(os.path.join('bunny_v2', 'bun000_v2_ptpl_paint045.ply'), ply1)
o3d.io.write_triangle_mesh(os.path.join('bunny_v2', 'bun045_v2_ptpl_reg_paint.ply'), ply2)
evaluation = evaluate_align(ply2, ply1)
print(evaluation)

[Open3D WARNING] Read PLY failed: unable to open file: bunny_v2/bun000_v2.ply
[Open3D WARNING] Read PLY failed: unable to open file: bunny_v2/bun045_v2.ply


RPly: Unable to open file
RPly: Unable to open file


RuntimeError: [1;31m[Open3D Error] (std::shared_ptr<PointCloud> open3d::geometry::TriangleMesh::SamplePointsUniformly(size_t, bool)) /Users/runner/work/Open3D/Open3D/cpp/open3d/geometry/TriangleMesh.cpp:526: Input mesh has no triangles.
[0;m